# Imports & Settings

In [1]:
import torch
torch.__version__
torch.cuda.is_available()

True

In [1]:
from torch_geometric.utils import (
    add_self_loops,
    negative_sampling,
    remove_self_loops ,
)
import torch.nn.functional as F
from sklearn.metrics import accuracy_score,roc_auc_score,average_precision_score
from sklearn.metrics import precision_score
import warnings
warnings.filterwarnings("ignore")
import time


import numpy as np
import scipy.sparse as sp
from torch import nn
import pandas as pd
# import networkx as nx
from typing import List , Dict,Tuple
from torch_geometric.nn import SAGEConv, to_hetero
import torch
from torch_geometric.loader import DataLoader

import matplotlib.pyplot as plt
import torch_geometric.nn as  gnn
from torch_geometric.data  import Data
from torch_geometric.data import HeteroData
import torch
import torch.nn as nn
%run "../src/week_graphs.py"
%run "../src/gnn.py"



### Data

In [2]:
batch_size=8

week_graphs = WeekGraphs("../../data/graphs/graph_dataset_title_rsi_indicator_volume")
train = week_graphs[:int(0.8*len(week_graphs))]
test = week_graphs[:int(len(week_graphs) - (0.8*len(week_graphs)))]
train_loader = DataLoader(train, batch_size=8)
test_loader = DataLoader(test, batch_size=8, shuffle=False)

In [3]:
len(train), len(test)

(1020, 255)

### Technical Indicators

In [3]:
train[0]

HeteroData(
  target=[310],
  missing_prices=[310],
  info_exists=[310],
  company_timeseries=[310, 15, 2],
  date=2015-01-28 00:00:00,
  company={ x=[310] },
  article={ x=[235, 768] },
  industry={ x=[12] },
  (article, main_company, company)={ edge_index=[2, 235] },
  (article, mentioned, company)={ edge_index=[2, 395] },
  (article, in_industry, industry)={ edge_index=[2, 235] },
  (company, mentioned_in, article)={ edge_index=[2, 395] },
  (company, in_industry, industry)={ edge_index=[2, 310] },
  (industry, has_company, company)={ edge_index=[2, 310] }
)

In [5]:
import wandb

from   tqdm import tqdm
# wandb.init(
#         project="test_sage",
#         name="run_historical_transformer_title_rsi_indicator_volume",
#         mode="online")

n_epochs = 50
# # warmup_steps=

neurostock = NeuroStock(node_emb_size=128, company_emb_size=64, graph_metadata=train[0].metadata(), lstm=False, num_timeseries_features=2)

neurostock.to('cpu')
device = next(neurostock.parameters()).device
optimizer =  torch.optim.AdamW(neurostock.parameters(), lr=0.001)
# lr_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.1,  total_iters=warmup_steps)
for e in tqdm(range(n_epochs)):
    train_losses= []
    neurostock.train()
    train_outs = []
    train_targets = []
    for batch  in train_loader:
      #with torch.autocast(device_type=device, dtype=torch.float16):
        batch = batch.to(device)
        out = neurostock(batch)
        train_outs.append(out.cpu().detach().reshape(-1))
        train_targets.append(batch["target"].cpu().detach().reshape(-1))
        loss = neurostock.compute_loss(out, batch["target"])
        optimizer.zero_grad()
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()
        # lr_scheduler.step()

    neurostock.eval()
    valid_losses = []
    # continue
    valid_outs = []
    valid_targets = []
    with torch.no_grad():
        for batch  in test_loader:
          batch = batch.to(device)
          out = neurostock(batch)
          valid_outs.append(out.reshape(-1).cpu().detach())
          valid_targets.append(batch["target"].reshape(-1).cpu().detach())
          loss = neurostock.compute_loss(out, batch["target"])
          valid_losses.append(loss.item())

    valid_outs = torch.cat(valid_outs).numpy()
    valid_targets = torch.cat(valid_targets).numpy()
    valid_acc = ((valid_outs >= 0.5) == valid_targets).mean()
    train_outs = torch.cat(train_outs).numpy()
    train_targets = torch.cat(train_targets).numpy()
    train_acc = ((train_outs >= 0.5) == train_targets).mean()
#     wandb.log({
#           "train_loss" : np.mean(train_losses),
#           "valid_loss" : np.mean(valid_losses),
#           "valid_acc" : valid_acc
#     })
# wandb.finish()


  2%|▏         | 1/50 [01:01<50:09, 61.41s/it]


KeyboardInterrupt: 

In [ ]:
# real_targets = torch.cat(real_targets).numpy()
acc = ((outs > 0.5) == real_targets).mean()
acc

0.5205866507033822

In [ ]:
sum(out.isnan())

tensor([472])

In [ ]:
range(int(len(dataset) * 0.8)), range(int(len(dataset) * 0.8), len(dataset))

In [17]:
stock_df = pd.read_csv("/content/drive/MyDrive/Stock Market Prediction Graduation/processed_stock_df.csv")

In [18]:
stock_df

,Unnamed: 0,symbol,<PER>,dateOfPrice,<TIME>,open,high,low,close,volume,<OPENINT>
0,2368,CVV,D,2014-12-26,0,14.80000,14.8480,14.21000,14.22000,1.210800e+04,0
1,4007,NVDA,D,2014-12-26,0,4.94082,4.9721,4.92834,4.94531,5.479256e+06,0
2,2475,HAIN,D,2014-12-26,0,58.81500,59.3300,58.57500,59.26000,4.231780e+05,0
3,2475,ENTG,D,2014-12-26,0,12.94190,12.9998,12.87410,12.89400,3.584945e+05,0
4,2475,COLM,D,2014-12-26,0,41.15110,41.5683,40.97800,41.22520,3.083397e+05,0
...,...,...,...,...,...,...,...,...,...,...,...
415704,3770,HST,D,2020-02-20,0,16.81500,16.8150,16.18490,16.66460,1.091273e+07,0
415705,3770,CME,D,2020-02-20,0,191.30800,192.5290,188.29600,192.26200,1.743956e+06,0
415706,4932,MRVL,D,2020-02-20,0,24.80020,25.1233,24.38520,24.77020,5.862125e+06,0
415707,9613,MAT,D,2020-02-20,0,13.05000,13.2850,13.01000,13.25000,4.558237e+06,0
